In [1]:
import numpy as np                        # numerical computations
from matplotlib import pyplot as plt      # visualisation
import xarray as xr                       # for netcdf
import pandas as pd                       
import os

In [2]:
first_file = xr.open_dataset("../data/Archivos_reanalisisERA5_realesINUMET/Reanalisis/Datos_t2m_horario_2000a2010_uy.nc")
first_file.head()

<xarray.Dataset>
Dimensions:    (longitude: 5, latitude: 5, time: 5)
Coordinates:
  * longitude  (longitude) float32 -58.0 -57.75 -57.5 -57.25 -57.0
  * latitude   (latitude) float32 -30.0 -30.25 -30.5 -30.75 -31.0
  * time       (time) datetime64[ns] 2000-01-01 ... 2000-01-01T04:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-12-13 16:08:08 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [11]:
def prepare_data(file):
    df = pd.DataFrame()

    df["time"] = pd.to_datetime(file.time)
    df["t2m"] = file.t2m.mean(dim=['latitude', 'longitude'])
    df.set_index("time")

    # type of the date column -> datetime64[ns] !!
    # separating the time column
    df["date"] = df["time"].dt.date
    df["day"] = df["time"].dt.day
    df["month"] = df["time"].dt.month

    # changing the format of the date column by removing the hours

    # in .groupby only 2 columns date and t2m are taken from the groupings
    # in .agg the dict is used to apply those functions only to t2m column
    df_by_day = df.groupby("date")[["date", "t2m"]].agg({"t2m": ["mean", "min", "max"]}).rename(columns={"t2m":"t2m_daily"})   # temp precipitation becomes daily and not hourly mesured

    df_by_day.columns = ['_'.join(col).strip() for col in df_by_day.columns.values]
    
    df = df.join(df_by_day, on="date").drop("date", axis = 1)
    
    return df

f1 = prepare_data(first_file)

In [6]:
second_file = xr.open_dataset("../data/Archivos_reanalisisERA5_realesINUMET/Reanalisis/Datos_t2m_horario_2011a2021_uy.nc")
second_file.head()

<xarray.Dataset>
Dimensions:    (longitude: 5, latitude: 5, time: 5)
Coordinates:
  * longitude  (longitude) float32 -58.0 -57.75 -57.5 -57.25 -57.0
  * latitude   (latitude) float32 -30.0 -30.25 -30.5 -30.75 -31.0
  * time       (time) datetime64[ns] 2011-01-01 ... 2011-01-01T04:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-12-12 19:36:09 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [12]:
f2 = prepare_data(second_file)

In [19]:
f1.append(f2) # cette ligne est ok, mais il faudrait modifier le append pour un concat

/var/folders/p8/csdc56jd2x5d40s91zt0pjh40000gn/T/ipykernel_44484/3638450512.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  f1.append(f2)


,time,t2m,day,month,t2m_daily_mean,t2m_daily_min,t2m_daily_max
0,2000-01-01 00:00:00,295.123260,1,1,295.357391,289.210754,301.250488
1,2000-01-01 01:00:00,294.918762,1,1,295.357391,289.210754,301.250488
2,2000-01-01 02:00:00,294.258972,1,1,295.357391,289.210754,301.250488
3,2000-01-01 03:00:00,291.075500,1,1,295.357391,289.210754,301.250488
4,2000-01-01 04:00:00,290.787567,1,1,295.357391,289.210754,301.250488
...,...,...,...,...,...,...,...
96427,2021-12-31 19:00:00,306.261017,31,12,300.427338,293.982544,306.261017
96428,2021-12-31 20:00:00,305.979279,31,12,300.427338,293.982544,306.261017
96429,2021-12-31 21:00:00,305.094940,31,12,300.427338,293.982544,306.261017
96430,2021-12-31 22:00:00,303.514465,31,12,300.427338,293.982544,306.261017
